In [ ]:
!apt install -qq chromium-chromedriver

The following additional packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 libudev1 snapd squashfs-tools systemd-hwe-hwdb
  udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver libfuse3-3 liblzo2-2 snapd squashfs-tools
  systemd-hwe-hwdb udev
The following packages will be upgraded:
  libudev1
1 upgraded, 9 newly installed, 0 to remove and 48 not upgraded.
Need to get 28.5 MB of archives.
After this operation, 118 MB of additional disk space will be used.
Preconfiguring packages ...
Selecting previously unselected package apparmor.
(Reading database ... 123597 files and directories currently installed.)
Preparing to unpack .../apparmor_3.0.4-2ubuntu2.3build2_amd64.deb ...
Unpacking apparmor (3.0.4-2ubuntu2.3build2) ...
Selecting previously unselected package liblzo2-2:amd64.
Preparing to unpack .../liblzo2-2_2.10-2build3_amd64.de

In [ ]:
!pip install -qq selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00


In [ ]:
# selenium import
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

# other imports
import os
import json
from tqdm import tqdm

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument('--headless') # must options for Google Colab
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")

In [ ]:
with open('/content/drive/MyDrive/AI4E/NLP_RAG_Project/data/crawl_mcq_data/links.json', 'r') as fIn:
  links = json.load(fIn)

In [ ]:
links

{'Hạng A1': 'https://lythuyet.onthigplx.vn/200-cau-hoi-dung-cho-sat-hach-lai-xe-mo-to-hang-a1/',
 'Hạng A2': 'https://lythuyet.onthigplx.vn/450-cau-hoi-dung-cho-sat-hach-lai-xe-mo-to-hang-a2/',
 'Hạng A3, A4': 'https://lythuyet.onthigplx.vn/500-cau-hoi-dung-cho-sat-hach-lai-xe-hang-a3-a4/',
 'Hạng B1': 'https://lythuyet.onthigplx.vn/574-cau-hoi-dung-cho-sat-hach-lai-xe-hang-b1/',
 'Hạng B2, C, D, E,F 1': 'https://lythuyet.onthigplx.vn/166-cau-hoi-ve-khai-niem-va-quy-tac-giao-thong-duong-bo/',
 'Hạng B2, C, D, E,F 2': 'https://lythuyet.onthigplx.vn/26-cau-hoi-ve-nghiep-vu-van-tai/',
 'Hạng B2, C, D, E,F 3': 'https://lythuyet.onthigplx.vn/21-cau-hoi-ve-van-hoa-dao-duc-nguoi-lai-xe/',
 'Hạng B2, C, D, E,F 4': 'https://lythuyet.onthigplx.vn/54-cau-hoi-ve-ky-thuat-lai-xe/',
 'Hạng B2, C, D, E,F 5': 'https://lythuyet.onthigplx.vn/35-cau-hoi-ve-cau-tao-va-sua-chua-xe/'}

## Thử nghiệm

In [ ]:
driver = webdriver.Chrome(options=chrome_options)


In [ ]:
driver.get(links["Hạng B2, C, D, E,F 5"])

In [ ]:
questions = driver.find_elements(By.XPATH, "//p[strong[contains(text(), 'Câu')]]")

In [ ]:
len(questions)

35

In [ ]:
questions[0].text

'Câu 6. Người lái xe được hiểu như thế nào trong các khái niệm dưới đây?'

In [ ]:
answers = driver.find_elements(By.XPATH, "//ul[li/input[@type='checkbox']]")

In [ ]:
choice_1 = answers[0].find_elements(By.TAG_NAME, "li")
correct_answer = [option.text for option in choice_1 if option.find_element(By.TAG_NAME, 'input').get_attribute('checked') is not None]

In [ ]:
i = 0
for choice in choice_1:
  i += 1
  print(f"{i}. {choice.text}")
print(correct_answer[0])

1. Là người điều khiển xe cơ giới
2. Là người điều khiển xe thô sơ.
3. Là người điều khiển xe có súc vật kéo.
Là người điều khiển xe cơ giới


In [ ]:
driver.close()

## Chạy thật

In [ ]:
results = []
def get_mcqs(driver, link):
  driver.get(link)
  questions = questions = driver.find_elements(By.XPATH, "//p[strong[contains(text(), 'Câu')]]")
  answers = driver.find_elements(By.XPATH, "//ul[li/input[@type='checkbox']]")

  for question, answer in zip(questions, answers):
    # Bỏ qua câu hỏi có hình ảnh
    if question.find_elements(By.TAG_NAME, 'img'):
        continue

    question_text = question.text
    options = answer.find_elements(by=By.TAG_NAME, value="li")
    choices = [option.text for option in options]
    correct_answer = [option.text for option in options if option.find_element(By.TAG_NAME, 'input').get_attribute('checked') is not None]

    results.append({
        "Câu hỏi": question_text,
        "Các lựa chọn": choices,
        "Đáp án đúng": correct_answer
    })


In [ ]:
driver = webdriver.Chrome(options=chrome_options)
for (level, link) in links.items():
  print(f"Processing {level}")
  get_mcqs(driver, link)
driver.close()

Processing Hạng A1
Processing Hạng A2
Processing Hạng A3, A4
Processing Hạng B1
Processing Hạng B2, C, D, E,F 1
Processing Hạng B2, C, D, E,F 2
Processing Hạng B2, C, D, E,F 3
Processing Hạng B2, C, D, E,F 4
Processing Hạng B2, C, D, E,F 5


In [ ]:
# Loại bỏ các câu trùng lặp giữa các hạng giấy phép
new_results = []
for result in results:
  if result not in new_results:
    new_results.append(result)

In [ ]:
len(new_results)

279

In [ ]:
result_path = '/content/drive/MyDrive/AI4E/NLP_RAG_Project/data/crawl_mcq_data/crawled_mcqs.json'
with open(result_path, 'w') as fOut:
  json.dump(new_results, fOut, indent=4, ensure_ascii=False)